<a href="https://colab.research.google.com/github/fabioPatino/Proyecto-individual-/blob/main/Limpieza/etl_review.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers
!pip install torch


In [8]:
# Importar las bibliotecas necesarias
import ast
import gzip
import pandas as pd

# Inicializar una lista vacía para almacenar los datos
data_list = []

# Definir la ruta del archivo comprimido en formato gzip que contiene los datos JSON
file_path = 'user_reviews.json.gz'

# Abrir el archivo gzip en modo lectura de texto con codificación UTF-8
with gzip.open(file_path, 'rt', encoding='utf-8') as f:
    # Procesar cada línea del archivo
    for line in f:
        try:
            # Usar ast.literal_eval para convertir la línea en un diccionario
            json_data = ast.literal_eval(line)
            # Añadir el diccionario a la lista de datos
            data_list.append(json_data)
        except ValueError as e:
            # Imprimir un mensaje de error si la línea no se puede convertir
            print(f"Error en la línea: {line}")
            continue

# Crear un DataFrame de pandas a partir de la lista de datos
pre_data = pd.DataFrame(data_list)


In [7]:
# Explota la columna 'reviews' en el DataFrame 'pre_data',
# de modo que cada elemento de la lista en 'reviews' se convierta en una fila separada.
data1 = pre_data.explode(['reviews'])

# Normaliza los datos JSON de la columna 'reviews' en 'data1',
# transformando los datos anidados en columnas planas.
# Luego, establece el índice de 'data2' para que coincida con el índice original de 'data1'.
data2 = pd.json_normalize(data1['reviews']).set_index(data1['reviews'].index)

# Concatenar 'data1' y 'data2' a lo largo del eje de columnas (axis=1),
# fusionando las columnas originales con las nuevas columnas extraídas de 'reviews'.
data = pd.concat([data1, data2], axis=1)

# Eliminar columnas innecesarias del DataFrame resultante 'data'.
# Aquí se eliminan las columnas 'reviews', 'posted', 'last_edited', 'helpful',
# 'recommend', 'user_url' y 'funny'.
data = data.drop(['reviews','posted','last_edited','helpful','recommend','user_url','funny'], axis=1, inplace=False)

# Mostrar las primeras 10 filas del DataFrame final.
data.head(10)


,user_id,item_id,review
0,76561197970982479,1250,Simple yet with great replayability. In my opi...
0,76561197970982479,22200,It's unique and worth a playthrough.
0,76561197970982479,43110,Great atmosphere. The gunplay can be a bit chu...
1,js41637,251610,I know what you think when you see this title ...
1,js41637,227300,For a simple (it's actually not all that simpl...
1,js41637,239030,Very fun little game to play when your bored o...
2,evcentric,248820,A suitably punishing roguelike platformer. Wi...
2,evcentric,370360,"""Run for fun? What the hell kind of fun is that?"""
2,evcentric,237930,"Elegant integration of gameplay, story, world ..."
2,evcentric,263360,"Random drops and random quests, with stat poin..."


In [6]:
data.to_parquet('data.parquet')

# Senitment Analisys

In [9]:
#
from tqdm import tqdm
from nltk.sentiment.vader import SentimentIntensityAnalyzer

# Descargar los recursos necesarios para VADER
import nltk
nltk.download('vader_lexicon')

# Función para análisis de sentimientos usando VADER
def analyze_sentiment(review):
    sid = SentimentIntensityAnalyzer()
    sentiment_scores = sid.polarity_scores(review)
    if sentiment_scores['compound'] >= 0.05:
        return 2  # Positivo
    elif sentiment_scores['compound'] <= -0.05:
        return 0  # Negativo
    else:
        return 1  # Neutral

# Leer el archivo .parquet
df = pd.read_parquet('data.parquet')

# Inicializar una lista para almacenar los sentimientos
sentiments = []

# Iterar sobre cada valor de la columna 'review'
for review in tqdm(df['review']):
    if isinstance(review, str):  # Verificar si el valor es una cadena de texto
        if review.strip() == '':
            sentiments.append(1)  # Valor '1' si la reseña está vacía o ausente
        else:
            sentiment_label = analyze_sentiment(review)
            sentiments.append(sentiment_label)
    elif isinstance(review, float) and pd.isnull(review):  # Manejar valores NaN
        sentiments.append(1)  # Valor '1' si el valor es NaN
    else:
        sentiments.append(1)  # Valor '1' por defecto para cualquier otro tipo de valor

# Asignar la lista de sentimientos al DataFrame en una nueva columna 'sentiment_analysis'
df['sentiment_analysis'] = sentiments

# Mapear los valores de sentimiento a la escala deseada ('0' para malo, '1' para neutral y '2' para positivo)
df['sentiment_analysis'] = df['sentiment_analysis'].map({0: 0, 1: 1, 2: 2, 3: 2, 4: 2})

# Eliminar la columna original 'review' y renombrar la nueva columna 'sentiment_analysis' a 'review'
df.drop(columns=['review'], inplace=True)
df.rename(columns={'sentiment_analysis': 'review'}, inplace=True)

# Guardar el DataFrame actualizado en un nuevo archivo CSV
df.to_csv('user_reviews_sentiment.csv', index=False)

# Mostrar las primeras filas del DataFrame con la columna de sentimiento aplicada
print(df.head())

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
100%|██████████| 59333/59333 [06:10<00:00, 160.12it/s]


             user_id item_id  review
0  76561197970982479    1250       2
0  76561197970982479   22200       2
0  76561197970982479   43110       2
1            js41637  251610       2
1            js41637  227300       2
